In [ ]:
import os
import urllib.request
import zipfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
dataset_path = "../data/download/"
data_raw = "../data/raw/"


def download_dataset(dataset_path):
    urllib.request.urlretrieve(
        "https://go.criteois.com/criteo-research-attribution-dataset.zip",
        dataset_path + "criteo_attribution_dataset.zip",
    )

    zippath = dataset_path + "criteo_attribution_dataset.zip"
    with zipfile.ZipFile(zippath, "r") as zip_ref:
        zip_ref.extractall(data_raw)


if not os.path.exists(data_raw + "criteo_attribution_dataset.tsv.gz"):
    print("Downloading dataset...")
    download_dataset(dataset_path)